# Fault Diagnosis Toolbox in Python -- a development notebook

Erik Frisk<br>
<frisk@isy.liu.se><br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

In [1]:
%matplotlib notebook
import sys
import csv
sys.path.append("faultdiagnosistoolbox/build/lib.macosx-10.6-x86_64-2.7/")
import numpy as np
import scipy.sparse as sp
import structuralanalysis as sa
import faultdiagnosistoolbox.dmperm as dmperm
import faultdiagnosistoolbox as fdt
import matplotlib
import matplotlib.pyplot as plt
dir(sa)

['__doc__',
 '__file__',
 '__name__',
 '__package__',
 'dmperm_internal',
 'findmso_internal']

## Some basic helper functions

In [2]:
def ReadMatrixCSV(fileName):
    with open(fileName, 'rt') as csvfile:
        matReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        size = matReader.next() # Assume first line contains size information
        X=np.zeros((int(size[0]),int(size[1])),dtype='float64');
        for edge in matReader:
            X[int(edge[0]),int(edge[1])] = float(edge[2])
        return sp.csc_matrix(X.astype(np.int64))
def BoxOff(*argin):
    if len(argin)>0:
        ax=argin[0]
    else:
        ax=plt.gca();
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

## Low level function calls

In [3]:
X=ReadMatrixCSV('Xengine.txt')
dm_int=sa.dmperm_internal(dmperm.CSCDict(X))

In [ ]:
print dm_int["p"]

In [ ]:
plt.figure(10)
plt.subplot(1,2,1)
plt.spy(X, marker="o", markersize=4);
plt.subplot(1,2,2)
plt.spy(X[dm_int["p"],:][:,dm_int["q"]], marker="o", markersize=4);

In [ ]:
plt.figure(11)
fdt.PlotDM(X,verbose=False)
plt.show()

## Try some basic dmperm functionality  and time performance

In [ ]:
reload(dmperm);

In [ ]:
dm=dmperm.GetDMParts(X)
dmperm.sprank(X)

Evaluate timing performance by doing 1000 dmperm 

Matlab comparisson (my iMac):<br>
tic; for k=1:1000; GetDMParts(X); end; toc<br>
Elapsed time is 0.108608 seconds.


In [ ]:
import timeit
timeit.timeit('dmperm.GetDMParts(X)', setup="from __main__ import dmperm, X",number=1000)

In [ ]:
print dmperm.IsPSO(X)
print dmperm.IsPSO(X[dm.Mp.row,:][:,dm.Mp.col])

In [ ]:
Xp=X[dm.Mp.row,:][:,dm.Mp.col]
pd=dmperm.PSODecomposition(Xp)

In [ ]:
print len(pd['eqclass'])
print len(pd['X0'])

## Try some MSO calculations

In [ ]:
msos=sa.findmso_internal(dmperm.CSCDict(X));
print len(msos)
print len(msos[0])
print msos[0]

In [ ]:
import timeit
timeit.timeit('sa.findmso_internal(dmperm.CSCDict(X))', setup="from __main__ import sa, dmperm, X",number=100)

In [ ]:
mso_n = map(lambda m:len(m), msos)
plt.figure(20)
plt.hist(mso_n,bins=10)
plt.title('Histogram - mso sizes')
plt.xlabel('MSO size')
plt.ylabel('Number of MSO sets')
BoxOff()
plt.show()

In [ ]:
msos[0]